### ***전처리 목표***
- 자사 vs 경쟁사 분석용 베이스 데이터 만들기
- 리뷰 수/방문자 리뷰수/블로그 리뷰수를 지점 단위 1행으로 정리
- 중복, 결측치 제거
- 로그 변환까지 적용
#### **즉, 분석 가능한 최소 단위 데이터 테이블을 만드는 단계**

In [1]:
# 0. 라이브러리
import pandas as pd
import numpy as np

In [2]:
# 1. 정제 완료 데이터 로드
금별 = pd.read_csv(
    r"C:\Users\user\Desktop\데이터분석\정제\금별맥주_정제완료.csv"
)

역전 = pd.read_csv(
    r"C:\Users\user\Desktop\데이터분석\정제\역전할머니_정제완료.csv"
)

In [3]:
# 자사 / 경쟁사 구분 변수 생성
금별["brand_type"] = "자사"
역전["brand_type"] = "경쟁사"

In [4]:
# 베이스 데이터 결합 (지점 단위)
df_base = pd.concat([금별, 역전], ignore_index=True)

# 이 시점의 df_base는 지점 정보만 있음

In [6]:
# 2. 리뷰 수 파일 로드
# 금별/역전 리뷰 수 파일 로드
금별_일반 = pd.read_csv(r"C:\Users\user\Desktop\데이터분석\수집 후 출력(CSV)\금별_리뷰수.csv")
금별_방문자 = pd.read_csv(r"C:\Users\user\Desktop\데이터분석\수집 후 출력(CSV)\금별_방문자리뷰수.csv")
금별_블로그 = pd.read_csv(r"C:\Users\user\Desktop\데이터분석\수집 후 출력(CSV)\금별_블로그리뷰수.csv")

역전_일반 = pd.read_csv(r"C:\Users\user\Desktop\데이터분석\수집 후 출력(CSV)\역전_리뷰수.csv")
역전_방문자 = pd.read_csv(r"C:\Users\user\Desktop\데이터분석\수집 후 출력(CSV)\역전_방문자리뷰수.csv")
역전_블로그 = pd.read_csv(r"C:\Users\user\Desktop\데이터분석\수집 후 출력(CSV)\역전_블로그리뷰수.csv")

In [7]:
# 컬럼명 정리
for df in [금별_일반, 금별_방문자, 금별_블로그, 역전_일반, 역전_방문자, 역전_블로그]:
    df.columns = df.columns.str.strip().str.replace(" ", "_")

In [8]:
# 3. 브랜드별 리뷰 통합 (place_id 기준)
# 금별
금별_리뷰 = 금별_일반.merge(
    금별_방문자[["place_id","방문자_리뷰수"]],
    on="place_id", how="outer"
).merge(
    금별_블로그[["place_id","블로그_리뷰수"]],
    on="place_id", how="outer"
)
금별_리뷰["brand_type"] = "자사"

In [9]:
# 역전
역전_리뷰 = 역전_일반.merge(
    역전_방문자[["place_id","방문자_리뷰수"]],
    on="place_id", how="outer"
).merge(
    역전_블로그[["place_id","블로그_리뷰수"]],
    on="place_id", how="outer"
)
역전_리뷰["brand_type"] = "경쟁사"

In [10]:
# 4. 리뷰 데이터 통합
리뷰 = pd.concat([금별_리뷰, 역전_리뷰], ignore_index=True)

In [11]:
# 5. 지점 단위 집계 (중복 제거)
리뷰_집계 = 리뷰.groupby("place_id", as_index=False).agg({
    "지점명":"first",
    "리뷰수":"max",
    "방문자_리뷰수":"max",
    "블로그_리뷰수":"max",
    "brand_type":"first"
})

In [12]:
# 6. df_base와 merge
df = df_base.merge(
    리뷰_집계[["place_id","리뷰수","방문자_리뷰수","블로그_리뷰수"]],
    on="place_id",
    how="left"
)

In [13]:
# 7. 리뷰 결측치 처리
df[["리뷰수","방문자_리뷰수","블로그_리뷰수"]] = df[["리뷰수","방문자_리뷰수","블로그_리뷰수"]].fillna(0)

In [14]:
# 8. 리뷰 수 0 지점 제거 (선택)
df = df[(df["리뷰수"]>0) | (df["방문자_리뷰수"]>0) | (df["블로그_리뷰수"]>0)]

In [15]:
# 9. 로그 변환 (분포 비대칭 완화)
df["log_리뷰수"] = np.log1p(df["리뷰수"])
df["log_방문자_리뷰수"] = np.log1p(df["방문자_리뷰수"])
df["log_블로그_리뷰수"] = np.log1p(df["블로그_리뷰수"])

In [16]:
# 10. 결과 확인
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 41 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   상가업소번호       430 non-null    object 
 1   brand        430 non-null    object 
 2   상호명          430 non-null    object 
 3   지점명          277 non-null    object 
 4   상권업종대분류코드    430 non-null    object 
 5   상권업종대분류명     430 non-null    object 
 6   상권업종중분류코드    430 non-null    object 
 7   상권업종중분류명     430 non-null    object 
 8   상권업종소분류코드    430 non-null    object 
 9   상권업종소분류명     430 non-null    object 
 10  표준산업분류코드     430 non-null    object 
 11  표준산업분류명      430 non-null    object 
 12  시도코드         430 non-null    int64  
 13  시도명          430 non-null    object 
 14  시군구코드        430 non-null    int64  
 15  시군구명         430 non-null    object 
 16  행정동코드        430 non-null    int64  
 17  행정동명         430 non-null    object 
 18  법정동코드        430 non-null    int64  
 19  법정동명    

,상가업소번호,brand,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,월간 검색량,블로그누적발행,encoded,brand_type,리뷰수,방문자_리뷰수,블로그_리뷰수,log_리뷰수,log_방문자_리뷰수,log_블로그_리뷰수
0,MA010120220804156426,금별맥주,금별맥주동양미래대점,동양미래대점,I2,음식,I211,주점,I21103,생맥주 전문,...,20.0,48.0,구로구 경인로47길 금별맥주,자사,160,267,58.0,5.081404,5.590987,4.077537
1,MA010120220808688996,금별맥주,금별맥주양천향교점,양천향교점,I2,음식,I211,주점,I21103,생맥주 전문,...,20.0,207.0,강서구 마곡동로10길 금별맥주,자사,218,244,47.0,5.389072,5.501258,3.871201
2,MA010120220806941672,금별맥주,금별맥주,NaN,I2,음식,I211,주점,I21103,생맥주 전문,...,20.0,133.0,송파구 법원로 금별맥주,자사,158,234,42.0,5.068904,5.459586,3.761200
3,MA010120220810164671,금별맥주,금별맥주송정역점,송정역점,I2,음식,I211,주점,I21103,생맥주 전문,...,20.0,24.0,강서구 송정로 금별맥주,자사,169,240,24.0,5.135798,5.484797,3.218876
4,MA010120220812201759,금별맥주,금별맥주,NaN,I2,음식,I211,주점,I21103,생맥주 전문,...,20.0,85.0,서대문구 연세로11길 금별맥주,자사,413,505,712.0,6.025866,6.226537,6.569481


In [18]:
df.to_csv(
    r"C:\Users\user\Desktop\데이터분석\분석용\Step1_전처리_자사경쟁사.csv",
    index=False,
    encoding="utf-8-sig"
)

### ***수행한 작업 요약***
| 단계 | 수행 내용                   | 결과                        |
| -- | ----------------------- | ------------------------- |
| 1  | 정제 완료 데이터 로드 (금별/역전)    | 지점 정보 + 브랜드 구분 변수 생성      |
| 2  | 리뷰 수 파일 로드 (일반/방문자/블로그) | 각 파일 컬럼명 정리               |
| 3  | 브랜드별 리뷰 통합              | place_id 기준으로 3개 리뷰 컬럼 합침 |
| 4  | 브랜드 통합                  | 자사/경쟁사 모두 합쳐서 리뷰 테이블 생성   |
| 5  | 지점 단위 집계                | 중복 제거 → 지점당 1행            |
| 6  | df_base와 merge          | 지점 정보 + 리뷰 수 통합           |
| 7  | 결측치 처리                  | 리뷰 수 없는 지점 → 0 처리         |
| 8  | 리뷰 0 지점 제거              | 비활성 지점 제외                 |
| 9  | 로그 변환                   | 리뷰 수 분포 비대칭 완화            |


### ***현재 df 상태***
- 단위: 지점 (place_id)
- 컬럼: 
place_id, 지점명, brand, brand_type, 리뷰수, 방문자_리뷰수, 블로그_리뷰수, log_리뷰수, log_방문자_리뷰수, log_블로그_리뷰수, ... (기타 지점 정보)
- 지점 수: 실제 분석 가능한 지점 수
- 자사/경쟁사 구분: brand_type
- 리뷰 수 결측치 없음, 로그 변환 완료

### ***의미***
1. 이제 Step 2 입지 변수 생성부터
    - 시군구 기준 경쟁률(요리주점 밀도)
    - 역과의 거리 등
    - 추가 파생 변수 생성 가능

2. 데이터는 분석 가능한 상태
    - 모든 지점이 1행
    - 성과 변수(리뷰 수) 정리 완료
    - 자사/경쟁사 구분 가능